# Intel Scene Classification - Train Fine Tune network

In [1]:
# Train the fine tuned network 
### Use 1000 training data and run for 50 epoch
### and store the model

from keras.models import load_model

## Load the trained new head model
head_model_file = './model/TrainHeadFull.hdf5'
model = load_model(head_model_file)

print(model)

Using Theano backend.


## UnFreeze some of the Conv layers for training

In [2]:
model.layers[75].trainable

False

In [3]:
len(model.layers)

177

In [4]:
for layer in model.layers[75:]:
    layer.trainable = True

In [5]:
model.layers[75].trainable

True

## Recompile the model

In [6]:
from keras.optimizers import SGD

In [7]:
opt = SGD(lr=0.001)

In [8]:
H = model.compile(loss="categorical_crossentropy", optimizer=opt,
                    metrics=["accuracy"])

In [9]:
model

In [10]:
#model.summary()

## Get Training Files

In [11]:
############################
## Get the training image file names
####################
from my_utils.datasets import DataSetLoader
import pandas as pd


DataLoader = DataSetLoader()
training_files = DataLoader.get_training_file_names()
training_labels = DataLoader.get_training_labels()

In [12]:
print(len(training_files))

17034


In [13]:
print(len(training_labels))

17034


In [14]:
training_files[0:5]

['dataset/train-scene classification/train/0.jpg',
 'dataset/train-scene classification/train/1.jpg',
 'dataset/train-scene classification/train/2.jpg',
 'dataset/train-scene classification/train/4.jpg',
 'dataset/train-scene classification/train/7.jpg']

In [15]:
training_labels[0:5]

[0, 4, 5, 0, 4]

## Transform the Labels

In [16]:
from sklearn.preprocessing import LabelBinarizer

In [17]:
label_binary = LabelBinarizer().fit_transform(training_labels)

## Split Train and Test Data

In [18]:
trainX = DataLoader.load_images(training_files[0:1000])

In [19]:
trainY = label_binary[0:1000]

In [20]:
valX = DataLoader.load_images(training_files[16900:17000])

In [21]:
valY = label_binary[16900:17000]

## Train the new model

In [22]:
model.fit(trainX, trainY, batch_size=25, validation_data=(valX, valY), 
          epochs=50,  verbose=1)

Train on 1000 samples, validate on 100 samples
Epoch 1/50
1000/1000 [==============================] - 1016s 1s/step - loss: 0.2458 - acc: 0.9150 - val_loss: 0.4119 - val_acc: 0.8600
Epoch 2/50
1000/1000 [==============================] - 1016s 1s/step - loss: 0.1535 - acc: 0.9480 - val_loss: 0.3185 - val_acc: 0.9100
Epoch 3/50
1000/1000 [==============================] - 1013s 1s/step - loss: 0.1204 - acc: 0.9600 - val_loss: 0.2974 - val_acc: 0.9300
Epoch 4/50
1000/1000 [==============================] - 1013s 1s/step - loss: 0.1155 - acc: 0.9610 - val_loss: 0.2988 - val_acc: 0.9300
Epoch 5/50
1000/1000 [==============================] - 1013s 1s/step - loss: 0.1094 - acc: 0.9650 - val_loss: 0.2978 - val_acc: 0.9300
Epoch 6/50
1000/1000 [==============================] - 1013s 1s/step - loss: 0.0896 - acc: 0.9790 - val_loss: 0.2991 - val_acc: 0.9200
Epoch 7/50
1000/1000 [==============================] - 1013s 1s/step - loss: 0.0783 - acc: 0.9820 - val_loss: 0.3018 - val_acc: 0.9200
E

In [23]:
model.save('./model/train_1000.hdf5')

## Evaluate Model

In [24]:
from sklearn.metrics import classification_report

In [25]:
preds = model.predict(valX)
print(classification_report(valY.argmax(axis=1), preds.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
           1       0.88      0.93      0.90        15
           2       0.80      0.73      0.76        22
           3       0.92      0.80      0.86        15
           4       0.86      1.00      0.93        19
           5       1.00      0.87      0.93        15

   micro avg       0.88      0.88      0.88       100
   macro avg       0.89      0.89      0.89       100
weighted avg       0.88      0.88      0.88       100

